

# **Tiki Web Scraping with Selenium**


**Overview**: Build a web-crawler that take in a Tiki URL and return a dataframe 


#Install resources

In [16]:
# install selenium and other resources for crawling data
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [17]:
### IMPORTS ###
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

#Configuration for Driver and links

In [18]:
###############
### GLOBALS ###
###############

# Header for chromedriver
HEADER = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'}
# Urls
TIKI                = 'https://tiki.vn'
MAIN_CATEGORIES = [
    {'Name': 'Điện Thoại - Máy Tính Bảng',
     'URL': 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner'},

    {'Name': 'Điện Tử - Điện Lạnh',
     'URL': 'https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner'},

    {'Name': 'Phụ Kiện - Thiết Bị Số', 
     'URL': 'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner'},

    {'Name': 'Laptop - Thiết bị IT', 
     'URL': 'https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner'},

    {'Name': 'Máy Ảnh - Quay Phim', 
     'URL': 'https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner'},

    {'Name': 'Điện Gia Dụng', 
     'URL': 'https://tiki.vn/dien-gia-dung/c1882?src=c.1882.hamburger_menu_fly_out_banner'},

    {'Name': 'Nhà Cửa Đời Sống', 
     'URL': 'https://tiki.vn/nha-cua-doi-song/c1883?src=c.1883.hamburger_menu_fly_out_banner'},

    {'Name': 'Hàng Tiêu Dùng - Thực Phẩm', 
     'URL': 'https://tiki.vn/bach-hoa-online/c4384?src=c.4384.hamburger_menu_fly_out_banner'},

    {'Name': 'Đồ chơi, Mẹ & Bé', 
     'URL': 'https://tiki.vn/me-va-be/c2549?src=c.2549.hamburger_menu_fly_out_banner'},

    {'Name': 'Làm Đẹp - Sức Khỏe', 
     'URL': 'https://tiki.vn/lam-dep-suc-khoe/c1520?src=c.1520.hamburger_menu_fly_out_banner'},

    {'Name': 'Thể Thao - Dã Ngoại', 
     'URL': 'https://tiki.vn/the-thao/c1975?src=c.1975.hamburger_menu_fly_out_banner'},

    {'Name': 'Xe Máy, Ô tô, Xe Đạp', 
     'URL': 'https://tiki.vn/o-to-xe-may-xe-dap/c8594?src=c.8594.hamburger_menu_fly_out_banner'},

    {'Name': 'Hàng quốc tế', 
     'URL': 'https://tiki.vn/hang-quoc-te/c17166?src=c.17166.hamburger_menu_fly_out_banner'},

    {'Name': 'Sách, VPP & Quà Tặng', 
     'URL': 'https://tiki.vn/nha-sach-tiki/c8322?src=c.8322.hamburger_menu_fly_out_banner'},

    {'Name': 'Voucher - Dịch Vụ - Thẻ Cào', 
     'URL': 'https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner'}
]

# Global driver to use throughout the script
DRIVER = None

In [4]:
MAIN_CATEGORIES[-1]['URL']

'https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner'

#Function to Start and Close Driver

In [19]:
# Function to (re)start driver
def start_driver(force_restart=False):
    global DRIVER
    
    if DRIVER is not None:
        if force_restart:
            DRIVER.close()
        else:
            raise RuntimeError('ERROR: cannot overwrite an active driver. Please close the driver before restarting.')
    
    # Setting up the driver
    options = webdriver.ChromeOptions()
    options.add_argument('-headless') # we don't want a chrome browser opens, so it will run in the background
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')

    DRIVER = webdriver.Chrome('chromedriver',options=options)

# Wrapper to close driver if its created
def close_driver():
    global DRIVER
    if DRIVER is not None:
        DRIVER.close()
    DRIVER = None

In [20]:
start_driver()

#Function to get info from one product

In [8]:
all_product_items = DRIVER.find_elements_by_class_name('product-item')
product_item = all_product_items[3]

In [9]:
product_item.get_attribute('href')

'https://tiki.vn/smart-tivi-casper-hd-32-inch-32hx6200-p73681047.html?__s=ssdp&itm_medium=CPC&itm_source=tiki-reco&itm_campaign=tiki-reco_UNK_DT_UNK_UNK_tiki-listing_UNK_p-category-mpid-listing-v1_20210929_MD_PID.73681047'

In [10]:
product_item.find_element_by_class_name('name').find_element_by_tag_name('span').get_attribute('innerHTML')

'Smart Tivi Casper HD 32 inch 32HX6200'

In [11]:
thumbnail = product_item.find_element_by_class_name('thumbnail')#YOUR CODE HERE
imgs = thumbnail.find_elements_by_tag_name('img')
img = imgs[1] if len(imgs) > 1 else imgs[0]

img.get_attribute('alt')

'Smart Tivi Casper HD 32 inch 32HX6200'

In [ ]:

d['name'] = product_item.find_element_by_class_name('name').text

In [21]:
#################
### FUNCTIONS ###
#################

# Function to extract product info from the product
def get_product_info_single(product_item):
    d = {'name':'',
         'price':'',
         'product_url':'',
         'image':'',
         'tiki_now':None,
         'free_delivery':None,
         'star':None,
         'badge_under_price':None,
         'discount_percentage':None,
         'shocking_price':None,
         'paid_by_installments':None,
         'free_gifts':None}
    # get thumbnail by class_name and Tag name and get_attribute()
    check_null = lambda x: True if x == '' else False 
    try:
        thumbnail = product_item.find_element_by_class_name('thumbnail')#YOUR CODE HERE
        imgs = thumbnail.find_elements_by_tag_name('img')
        img = imgs[1] if len(imgs) > 1 else imgs[0]
        d['image'] = img.get_attribute('src')

        free_delivery = imgs[0] if len(imgs) > 1 else None
        if free_delivery is not None:
          d['free_delivery'] = True if free_delivery.get_attribute('src') in ['https://salt.tikicdn.com/ts/upload/9b/8d/9a/1f3c2633f5f2530f85c8f59d8a816653.png','https://salt.tikicdn.com/ts/upload/dc/0d/49/3251737db2de83b74eba8a9ad6d03338.png'] else False
    except NoSuchElementException:
        pass

    # name get name through find_element_by_class_name
    try:
        #YOUR CODE HERE
        d['name'] = product_item.find_element_by_class_name('name').find_element_by_tag_name('span').get_attribute('innerHTML')
    except NoSuchElementException:
        pass

    # get price find_element_by_class_name
    try:
        #d['price'] = product_item.find_element_by_class_name('price-discount').text#YOUR CODE HERE
        #if check_null(d['price']):
        d['price'] = product_item.find_element_by_class_name('price-discount__price').get_attribute('innerHTML')
        d['discount_percentage'] = product_item.find_element_by_class_name('price-discount__discount').get_attribute('innerHTML')
    except NoSuchElementException:
        d['price'] = -1
    
    # get link from .get_attribute()
    try:
        product_link     = product_item.get_attribute('href')#YOUR CODE HERE
        d['product_url'] = product_link#YOUR CODE HERE CLEAN LINK WITH STRING MANUPULATION
    except NoSuchElementException:
        pass

    #tiki_now
    try:
      tiki_now = product_item.find_element_by_class_name('badge-service').find_element_by_tag_name('img').get_attribute('src')
      d['tiki_now'] = True if tiki_now == 'https://salt.tikicdn.com/ts/upload/9f/32/dd/8a8d39d4453399569dfb3e80fe01de75.png' else False
    except NoSuchElementException:
      pass

    try:
      badge_under_price = product_item.find_element_by_class_name('badge-under-price').find_element_by_tag_name('img').get_attribute('src')
      d['badge_under_price'] = True if badge_under_price == 'https://salt.tikicdn.com/ts/upload/51/ac/cc/528e80fe3f464f910174e2fdf8887b6f.png' else False
    except NoSuchElementException:
      pass

    try:
      badge_benefit = product_item.find_element_by_class_name('badge-benefits')
      paid_by_installments = badge_benefit.find_element_by_tag_name('img').get_attribute('src')
      d['paid_by_installments'] = True if paid_by_installments == 'https://salt.tikicdn.com/ts/upload/ba/4e/6e/26e9f2487e9f49b7dcf4043960e687dd.png' else False
    except NoSuchElementException:
      pass

    try:
      freegift_list = product_item.find_element_by_class_name('freegift-list').find_element_by_tag_name('img').get_attribute('src')
      d['free_gifts'] = True if freegift_list == 'https://salt.tikicdn.com/ts/upload/47/35/8c/446f61d046eba9a305d3f39dc0834c4a.png' else False
    except NoSuchElementException:
      pass

    return d


#Function to scrape info of all products from a Page URL

In [22]:
# Function to scrape all products from a page
def get_product_info_from_page(page_url):
    """ Extract info from all products of a specfic page_url on Tiki website
        Args:
            page_url: (string) url of the page to scrape
        Returns:
            data: list of dictionary of products info. If no products shown, return empty list.
    """
    global DRIVER
    n_skip=0
    n_success=0
    data = []
    DRIVER.get(page_url) # Use the driver to get info from the product page
    time.sleep(5) ## Must have the sleep function

    products_all = DRIVER.find_elements_by_class_name('product-item')#YOUR CODE HERE
    print(f'Found {len(products_all)} products')

    for product in products_all:
      #Look through the product and get the data
      # YOUR CODE HERE
      try:
        data.append(get_product_info_single(product))
        n_success+=1
      except NoSuchElementException:
        print('Cannot get this product. Skip')
        n_skip+=1
    print(f'Number of scraped products: {n_success}')
    print(f'Number of skipped products: {n_skip}')
    return data

In [23]:
######################
### START SCRAPING ###
######################
prod_data = []
num_max_page = 5

start_driver(force_restart=True)
#for main_cat_url in MAIN_CATEGORIES:
main_cat_url = MAIN_CATEGORIES[1]
#CODE TO GET DATA
for  i  in  range(num_max_page):
  page = main_cat_url['URL'].replace('?','?'+'page'+str(i)+'&')
  #print(page)
  prod_data = prod_data + get_product_info_from_page(page)

close_driver()

#SAVE DATA TO CSV FILE
df = pd.DataFrame(data=prod_data, columns=prod_data[0].keys())
df.to_csv('tiki_products.csv')


Found 72 products
Number of scraped products: 72
Number of skipped products: 0
Found 72 products
Number of scraped products: 72
Number of skipped products: 0
Found 72 products
Number of scraped products: 72
Number of skipped products: 0
Found 72 products
Number of scraped products: 72
Number of skipped products: 0
Found 72 products
Number of scraped products: 72
Number of skipped products: 0


In [24]:
df.head(10)

,name,price,product_url,image,tiki_now,free_delivery,star,badge_under_price,discount_percentage,shocking_price,paid_by_installments,free_gifts
0,Tủ lạnh Inverter Panasonic NR-BA189PAVN (167L)...,5.014.000 ₫,https://tka.tiki.vn/pixel?CLICK=&data=djAwMWzL...,https://salt.tikicdn.com/cache/200x200/ts/prod...,None,True,None,True,-24%,None,None,None
1,Tủ Lạnh Aqua 130 lít AQR-T150FA-BS,-1,https://tiki.vn/tu-tu-lanh-aqua-130-lit-aqr-t1...,https://salt.tikicdn.com/cache/200x200/ts/prod...,None,True,None,None,None,None,True,None
2,Tủ lạnh Samsung Inverter 208 lít RT19M300BGS/S...,4.699.000 ₫,https://tiki.vn/tu-lanh-samsung-inverter-208-l...,https://salt.tikicdn.com/cache/200x200/ts/prod...,True,True,None,True,-5%,None,True,None
3,Smart Tivi Casper HD 32 inch 32HX6200,-1,https://tiki.vn/smart-tivi-casper-hd-32-inch-3...,https://salt.tikicdn.com/cache/200x200/ts/prod...,False,True,None,None,None,None,True,None
4,Tủ lạnh Samsung Inverter 488 lít RF48A4000B4/S...,-1,https://tka.tiki.vn/pixel?CLICK=&data=djAwMaRg...,https://salt.tikicdn.com/cache/200x200/ts/prod...,True,True,None,None,None,None,True,None
5,Tủ lạnh Electrolux 85L EUM0900SA,-1,https://tiki.vn/tu-lanh-electrolux-85l-eum0900...,https://salt.tikicdn.com/cache/200x200/ts/prod...,None,True,None,None,None,None,None,None
6,Túi vệ sinh máy lạnh RO _ túi bảo dưỡng điều h...,-1,https://tiki.vn/tui-ve-sinh-may-lanh-ro-_-tui-...,https://salt.tikicdn.com/cache/200x200/ts/prod...,None,True,None,True,None,None,None,None
7,Tủ lạnh Sharp Inverter 165 lít SJ-X196E-DSS,-1,https://tiki.vn/tu-lanh-sharp-inverter-165-lit...,https://salt.tikicdn.com/cache/200x200/ts/prod...,None,None,None,None,None,None,True,None
8,Tủ lạnh Samsung Inverter 595 lít RS64T5F01B4/S...,-1,https://tka.tiki.vn/pixel?CLICK=&data=djAwMaDS...,https://salt.tikicdn.com/cache/200x200/ts/prod...,True,True,None,True,None,None,True,None
9,Smart Tivi Samsung 4K 50 inch UA50TU8100,12.390.000 ₫,https://tiki.vn/smart-tivi-samsung-4k-50-inch-...,https://salt.tikicdn.com/cache/200x200/ts/prod...,None,None,None,None,-20%,None,None,None
